RFM-analysis

Задача:сегментировать клиентов по группам (recency-давность покупки, freqency-частота покупок, monetary-общая сумма покупок)

Данные csv-файла:
в столбце order_date  - дата покупки
в столбце order_id	-ID клиента
в столбце customer  - инициалы клиента
в столбце grand_total  - сумма покупки

In [1]:
import pandas as pd                                                  # импортируем необходимые библиотеки
import datetime as dt

In [45]:
data=pd.read_csv('C:/Users/user/Documents/data/data.csv',sep=',')    # считываем файл csv в таблицу
data.info()                                                          # видим, что столбец с датой не в том формате, с которым можно проводить вычисления

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5009 entries, 0 to 5008
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   order_date   5009 non-null   object
 1   order_id     5009 non-null   object
 2   customer     5009 non-null   object
 3   grand_total  5009 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 156.7+ KB


In [46]:
data['order_date']=pd.to_datetime(data['order_date'])    # преобразуем столбец с датой в необходимый формат
display(data.head(5))                                    # выводим первые 5 строк таблицы
data.info()                                              # видим, что теперь столбец с датой в удобном для вычислений формате)
data.isna().sum()                                        # также проверяем, нет ли пустых значений в таблице(также это было видно и в предыдущем запросе)

,order_date,order_id,customer,grand_total
0,2011-09-07,CA-2011-100006,Dennis Kane,378
1,2011-07-08,CA-2011-100090,Ed Braxton,699
2,2011-03-14,CA-2011-100293,Neil Franz�sisch,91
3,2011-01-29,CA-2011-100328,Jasper Cacioppo,4
4,2011-04-08,CA-2011-100363,Jim Mitchum,21


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5009 entries, 0 to 5008
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   order_date   5009 non-null   datetime64[ns]
 1   order_id     5009 non-null   object        
 2   customer     5009 non-null   object        
 3   grand_total  5009 non-null   int64         
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 156.7+ KB


order_date     0
order_id       0
customer       0
grand_total    0
dtype: int64

In [47]:
print(data.max())                                    #теперь находим максимальные  значения по столбцам для дальнейшей работы, нам нужно знать дату последней покупки

now=dt.datetime(2014,12,31)                          #создаем дату новизны (самая свежая дата покупки), от которой будем отталкиваться в расчетах 2014-12-31

order_date     2014-12-31 00:00:00
order_id            US-2014-169551
customer        Zuschuss Donatelli
grand_total                  23661
dtype: object


In [48]:
RFMtable=data.groupby('customer').agg({'order_date':lambda x:(now-x.max()).days, # группируем данные по каждому клиенту и создаем столбцы-сколько прошло дней с последней покупки клиента (Recency)
                                       'order_id':lambda x:len(x),               # количество покупок клиента (Frequency)
                                       'grand_total':lambda x:x.sum()})          # общая сумма покупок клиента (Monetary)

RFMtable['order_date']=RFMtable['order_date'].astype(int)                        # преобразуем столбец с датой в численный тип, чтобы вычислить количество дней

RFMtable.rename(columns={'order_date':'recency',                                 # переименовываем столбцы для удобства,сохраняем их в данной таблице 
                        'order_id':'frequency',
                        'grand_total':'monetary'},inplace=True)

RFMtable


,recency,frequency,monetary
customer,,,
Aaron Bergman,415,3,887
Aaron Hawkins,12,7,1744
Aaron Smayling,88,7,3050
Adam Bellavance,54,8,7756
Adam Hart,34,10,3249
...,...,...,...
Xylona Preis,43,11,2375
Yana Sorensen,9,8,6721
Yoseph Carroll,4,5,5455


In [49]:
quantiles=RFMtable.quantile(q=[0.25,0.5,0.75])                   # делаем разбивку полученной таблицы по квантилям, у нас будет 4 подгруппы по каждому признаку RFM-анализа
quantiles

,recency,frequency,monetary
0.25,30.0,5.0,1145.0
0.50,75.0,6.0,2257.0
0.75,183.0,8.0,3784.0


In [67]:
quantiles=quantiles.to_dict()                          # преобразуем таблицу к словарю
quantiles

{'recency': {0.25: 30.0, 0.5: 75.0, 0.75: 183.0},
 'frequency': {0.25: 5.0, 0.5: 6.0, 0.75: 8.0},
 'monetary': {0.25: 1145.0, 0.5: 2257.0, 0.75: 3784.0}}

In [51]:
RFMsegment=RFMtable                          # создаем таблицу на основе RFMtable для дальнейшей работы

In [52]:
def RClass(x,p,d):                           # пишем функцию RClass, чтобы распределить клиентов по квантилям по столбцу recency; аргументы функции: х-value; p-recency, monetary value,frequency; d- квантили
    if x<=d[p][0.25]:
        return 1
    elif x<=d[p][0.5]:
        return 2
    elif x<=d[p][0.75]:
        return 3
    else:
        return 4

def FMClass(x,p,d):                           # пишем функцию FMClass, чтобы распределить клиентов по квантилям по столбцам frequency и monetary; аргументы функции: х-value, p-recency, monetary value,frequency, d- квантили
    if x<=d[p][0.25]:
        return 4
    elif x<=d[p][0.5]:
        return 3
    elif x<=d[p][0.75]:
        return 2
    else:
        return 1

In [53]:
RFMsegment['Rquartile']=RFMsegment['recency'].apply(RClass,args=('recency',quantiles,)) #добавляем столбец со значением нашей шкалы по столбцу recency, то есть применяем эти функции для нахождения значений по показателям у каждого покупателя
RFMsegment['Fquartile']=RFMsegment['frequency'].apply(FMClass,args=('frequency',quantiles,)) #аналогично проводим работу по столбцам frequency и monetary
RFMsegment['Mquartile']=RFMsegment['monetary'].apply(FMClass,args=('monetary',quantiles,))

RFMsegment



,recency,frequency,monetary,Rquartile,Fquartile,Mquartile
customer,,,,,,
Aaron Bergman,415,3,887,4,4,4
Aaron Hawkins,12,7,1744,1,2,3
Aaron Smayling,88,7,3050,3,2,2
Adam Bellavance,54,8,7756,2,2,1
Adam Hart,34,10,3249,2,1,2
...,...,...,...,...,...,...
Xylona Preis,43,11,2375,2,1,2
Yana Sorensen,9,8,6721,1,2,1
Yoseph Carroll,4,5,5455,1,4,1


In [55]:
#для удобства вводим столбец сразу с тремя характеристиками одновременно
RFMsegment['RFMClass']=RFMsegment.Rquartile.map(str)\
                                +RFMsegment.Fquartile.map(str)\
                                +RFMsegment.Mquartile.map(str)
RFMsegment

,recency,frequency,monetary,Rquartile,Fquartile,Mquartile,RFMClass
customer,,,,,,,
Aaron Bergman,415,3,887,4,4,4,444
Aaron Hawkins,12,7,1744,1,2,3,123
Aaron Smayling,88,7,3050,3,2,2,322
Adam Bellavance,54,8,7756,2,2,1,221
Adam Hart,34,10,3249,2,1,2,212
...,...,...,...,...,...,...,...
Xylona Preis,43,11,2375,2,1,2,212
Yana Sorensen,9,8,6721,1,2,1,121
Yoseph Carroll,4,5,5455,1,4,1,141


Непосредственно сегментация, то есть разделение клиентов по группам

In [56]:
RFMsegment[RFMsegment['RFMClass']=='111'].sort_values('monetary',ascending=False).head(6) # фильтруем только тех клиентов, у которых RFMClass=111, сортируем в порядке убывания по сумме покупок, выводим 6 значений

,recency,frequency,monetary,Rquartile,Fquartile,Mquartile,RFMClass
customer,,,,,,,
Sanjit Engle,9,11,12210,1,1,1,111
John Lee,21,11,9801,1,1,1,111
Pete Kriz,9,12,8647,1,1,1,111
Harry Marie,2,10,8237,1,1,1,111
Lena Creighton,16,12,7661,1,1,1,111
Patrick O'Brill,4,11,7474,1,1,1,111


In [65]:
RFMsegment['RFMClass'].value_counts()                   # находим количество клиентов каждой подгруппы RFMClass

444    67
443    37
144    31
344    31
111    30
       ..
134     2
331     2
424     2
412     2
114     1
Name: RFMClass, Length: 62, dtype: int64

Таким образом, мы сегментировали клиентов по подгруппам (делили на 4 группы по 3 признакам-recency(давность покупки),frequency(частота покупок),monetary(сумма покупок)) и нашли подгруппу клиентов, приносящих самую большую прибыль компании. Это подгруппа с показателем RFMClass=111(самая неболшая давность покупки, самая высокая частота покупок и самая большая сумма покупок)-их количество равно 30 человек из клиентской базы.
Аналогично можем увидеть и посчитать и другие сегментированные подгруппы.
Далее можно разработать дальнейшие стратегии для удержания и стимулирования необходимых подгрупп клиентов. Либо сделать акцент только на определенных подгруппах клиентов.